# Data Preparation

***import libraries***

In [ ]:
import http
import urllib.request
import json
import re
import pandas as pd
import openpyxl
import glob
import os
import shutil
import zipfile
import xml.etree.ElementTree as ET
import requests
import certifi
import ssl
from pathlib import Path

***mount google drive***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***input file of unknown ndcs***

In [ ]:
df_input = pd.read_excel("/content/drive/MyDrive/jw_jz_4.3.1/ndcs_python_05_2024.xlsx")

In [ ]:
len(df_extra)

1293

***merge with ndc crosswalk***

In [ ]:
df_crosswalk = pd.read_excel("/content/drive/MyDrive/jw_jz_4.3.1/NDC_SetID_Crosswalk.xlsx")

In [ ]:
df_extra =  pd.read_excel("/content/drive/MyDrive/jw_jz_4.3.1/extra_ndcs.xlsx")

In [ ]:
df_merged_input_extra = pd.merge(df_input, df_extra, how="inner", left_on="ndc", right_on="ndcpackagecode")

In [ ]:
merged_input_crosswalk_df = pd.merge(df_extra, df_crosswalk, how="inner", left_on="ndcpackagecode", right_on="DailyMed_NDC")

In [ ]:
len(merged_input_crosswalk_df)

799

***derive dailymed urls***

In [ ]:
setids = merged_input_crosswalk_df['setID'].to_list()

In [ ]:
urls = ["https://dailymed.nlm.nih.gov/dailymed/getFile.cfm?setid=" + setid + "&type=zip" for setid in setids]

In [ ]:
len(urls)

799

In [ ]:
 merged_input_crosswalk_df.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/dailymed_zip/june_2024/ merged_input_crosswalk_df.xlsx")

***download zip files***

In [ ]:
i = 0
j = 0
for url in urls:
 destination = "/content/drive/MyDrive/jw_jz_4.3.1/dailymed_zip/june_2024/" + url[56:-9] + ".zip"
 try:
        # Send a GET request to the URL
        response = requests.get(url)
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
          i = i + 1
          print(i)
            # Save the content to a file or process it as needed
          with open(destination, 'wb') as file:
            file.write(response.content)

            print(f"Content from {url} downloaded successfully.")
        else:
            print(f"Failed to download content from {url}. Status code: {response.status_code}")
            j = j + 1
            print(j)
 except Exception as e:
        print(f"An error occurred while downloading content from {url}: {str(e)}")

1
Content from https://dailymed.nlm.nih.gov/dailymed/getFile.cfm?setid=22ca78b4-f5a3-4144-cf89-5f633acf1e6d&type=zip downloaded successfully.
2
Content from https://dailymed.nlm.nih.gov/dailymed/getFile.cfm?setid=4d3b0076-3780-413f-9052-b23be5f18f01&type=zip downloaded successfully.
3
Content from https://dailymed.nlm.nih.gov/dailymed/getFile.cfm?setid=f12cea0d-b7cd-41d8-b551-2c74cc7f5f4e&type=zip downloaded successfully.
4
Content from https://dailymed.nlm.nih.gov/dailymed/getFile.cfm?setid=d38a1247-28c8-a911-e691-a983a2f9a6e3&type=zip downloaded successfully.
5
Content from https://dailymed.nlm.nih.gov/dailymed/getFile.cfm?setid=4d3b0076-3780-413f-9052-b23be5f18f01&type=zip downloaded successfully.
6
Content from https://dailymed.nlm.nih.gov/dailymed/getFile.cfm?setid=91ac4c14-8dd8-4efe-a462-d1c488a6184f&type=zip downloaded successfully.
7
Content from https://dailymed.nlm.nih.gov/dailymed/getFile.cfm?setid=5e5b0ed3-b104-45a7-bb3e-ee56febb46e8&type=zip downloaded successfully.
8
Cont

***download xml files***

In [ ]:
# Function to extract and rename XML files from zip archives
def extract_rename_xml(zip_folder, output_folder):
    i = 0
    j = 0
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through each zip file in the input folder
    for zip_file in os.listdir(zip_folder):
        if zip_file.endswith('.zip'):
            with zipfile.ZipFile(os.path.join(zip_folder, zip_file), 'r') as zip_ref:
                # Extract all files to a temporary directory
                temp_folder = os.path.join(output_folder, 'temp')
                zip_ref.extractall(temp_folder)

                # Find the XML file and rename it
                for file in os.listdir(temp_folder):
                    if file.endswith('.xml'):
                        i = i + 1
                        print(i)
                        xml_file = os.path.join(temp_folder, file)
                        new_name = os.path.splitext(zip_file)[0] + '.xml'
                        new_xml_path = os.path.join(output_folder, new_name)
                        shutil.move(xml_file, new_xml_path)

                # Remove the temporary folder
                shutil.rmtree(temp_folder)

# Specify the input folder containing zip files and the output folder
zip_folder = '/content/drive/MyDrive/jw_jz_4.3.1/dailymed_zip/june_2024'
output_folder = '/content/drive/MyDrive/jw_jz_4.3.1/dailymed_xml/june_2024'

# Call the function to extract and rename XML files
extract_rename_xml(zip_folder, output_folder)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# SDC Classification - REGEX#






 ***immunotherapy***

In [ ]:
def search_immunotherapy_in_folder(folder_path, output_excel_path):
    # Define the words to search for
    target_word_immunotherapy = "immunotherapy"
    target_words_how_supplied = ["injection", "infusion"]

    # Create an empty DataFrame to store the results
    results_df = pd.DataFrame(columns=['setid', 'immuno_flag'])

    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            xml_file_path = os.path.join(folder_path, filename)

            # Open and read the content of the XML file
            with open(xml_file_path, 'r', encoding='utf-8') as file:
                xml_content = file.read()

                # Check if immunotherapy word exists in the content
                found_immunotherapy = 1 if target_word_immunotherapy.lower() in xml_content.lower() else 0

                # Use regex to find the "How Supplied" section content
                pattern = re.compile(r'<title>.*HOW SUPPLIED.*?</title>\s*<text>(.*?)<title>', re.DOTALL | re.IGNORECASE)
                match = pattern.search(xml_content)

                # Check if the section was found and if it contains the target words for "How Supplied"
                found_how_supplied = 1 if match and any(word.lower() in match.group(1).lower() for word in target_words_how_supplied) else 0

                # Combine the results for both immunotherapy and how supplied
                found_combined = 1 if found_immunotherapy and found_how_supplied else 0

                # Append the results to the DataFrame
                results_df = results_df.append({'setid': filename, 'immuno_flag': found_combined}, ignore_index=True)

    # Save the results to an Excel file
    results_df.to_excel(output_excel_path, index=False)
    print(f"Results saved to {output_excel_path}")

# Provide the path to your folder containing XML files
xml_folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/XML_9000"

# Specify the output Excel file path
output_excel_path = "/content/drive/MyDrive/jw_jz_4.3.1/9000_immunotherapy_flag.xlsx"

# Call the function to search in the folder and save results to Excel
search_immunotherapy_in_folder(xml_folder_path, output_excel_path)

 ***how supplied***

In [ ]:
def process_how_supplied(xml_content):
    # Define the words to search for in "How Supplied" section
    target_words = ["injection", "infusion"]
    keyword_list = ["single-dose", "single dose", "single-use", "single use"]

    # Use regex to find the "How Supplied" section content
    pattern = re.compile(r'<title.*HOW SUPPLIED.*?</title>\s*<text>(.*?)<title>', re.DOTALL | re.IGNORECASE)
    match = pattern.search(xml_content)

    # Check if the section was found and if it contains the target words
    if match:
        how_supplied_content = match.group(1)
        result = any(word.lower() in how_supplied_content.lower() for word in target_words) and any(item.lower() in how_supplied_content.lower() for item in keyword_list)
        return 1 if result else 0
    else:
        return 0

def process_xml_file(xml_file_path):
    # Read the content of the XML file
    with open(xml_file_path, 'r', encoding='utf-8') as file:
        xml_content = file.read()

        # Process the "How Supplied" section and get the result
        sdc_status = process_how_supplied(xml_content)

    return {"setid": os.path.basename(xml_file_path), "supplied_flag": sdc_status}

def process_xml_folder(folder_path):
    results = []

    # Iterate through each file in the folder
    i = 0
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            xml_file_path = os.path.join(folder_path, filename)

            # Process each XML file and store the result in a list
            result = process_xml_file(xml_file_path)
            results.append(result)
            i = i + 1
            print(i)

    # Convert the list of results to a DataFrame
    df = pd.DataFrame(results)
    return df

# Provide the path to your folder containing XML files
xml_folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/XML_9000"

# Call the function to process the folder
df_results_supplied = process_xml_folder(xml_folder_path)

# Print the DataFrame
df_results_supplied.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/9000_supplied_flag.xlsx")

 ***multi dose***

In [ ]:
import os
import re
import pandas as pd

def check_display_and_how_supplied_in_folder(folder_path, output_excel_path):
    # Define the words to search for
    target_words = ["multiple-dose", "multiple dose", "multiple-use", "multiple use", "multi-dose", "multi dose", "multi-use", "multi use"]

    # Create an empty DataFrame to store the results
    results_df = pd.DataFrame(columns=['setid', 'multi'])

    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            xml_file_path = os.path.join(folder_path, filename)

            # Read the content of the XML file
            with open(xml_file_path, 'r', encoding='utf-8') as file:
                xml_content = file.read()

                # Use regex to find the "How Supplied" section content
                pattern_how_supplied = re.compile(r'<title>.*HOW SUPPLIED.*?</title>\s*<text>(.*?)<title>', re.DOTALL | re.IGNORECASE)
                match_how_supplied = pattern_how_supplied.search(xml_content)

                # Check if the "How Supplied" section was found and contains the target words
                if match_how_supplied and any(word.lower() in match_how_supplied.group(1).lower() for word in target_words):
                    result = '1'
                else:
                    # Use regex to find the "Display Panel" section content
                    pattern_display = re.compile(r'<title>.*DISPLAY PANEL.*?</title>\s*<text>(.*?)<title>', re.DOTALL | re.IGNORECASE)
                    match_display = pattern_display.search(xml_content)

                    # Check if the "Display Panel" section was found and contains the target words
                    result = '1' if match_display and any(word.lower() in match_display.group(1).lower() for word in target_words) else '0'

                # Append the results to the DataFrame
                results_df = results_df._append({'setid': filename, 'multi': result}, ignore_index=True)

    # Save the results to an Excel file
    results_df.to_excel(output_excel_path, index=False)
    print(f"Results saved to {output_excel_path}")

# Provide the path to your folder containing XML files
xml_folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/dailymed_xml/may_2024"

# Specify the output Excel file path
output_excel_path = "/content/drive/MyDrive/jw_jz_4.3.1/title_split/9000_multi_flag.xlsx"

# Call the function to check in the folder and save results to Excel
check_display_and_how_supplied_in_folder(xml_folder_path, output_excel_path)

Results saved to /content/drive/MyDrive/jw_jz_4.3.1/title_split/9000_multi_flag.xlsx


 ***improper administration***

In [ ]:
def check_how_supplied_in_folder(folder_path, output_excel_path):
    # Define the words to search for in the "How Supplied" section
    target_words_how_supplied = ["implant", "tablet", "single-patient-use", "single-patient use", "single patient use", "blister", "capsule", "pharmacy bulk", "inhalation", "oral", "gel"]

    # Create an empty DataFrame to store the results
    results_df = pd.DataFrame(columns=['setid', 'improper_admin'])

    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            xml_file_path = os.path.join(folder_path, filename)

            # Read the content of the XML file
            with open(xml_file_path, 'r', encoding='utf-8') as file:
                xml_content = file.read()

                # Use regex to find the "How Supplied" section content
                pattern_how_supplied = re.compile(r'<title>.*HOW SUPPLIED.*?</title>\s*<text>(.*?)<title>', re.DOTALL | re.IGNORECASE)
                match_how_supplied = pattern_how_supplied.search(xml_content)

                # Check if the "How Supplied" section was found and contains the target words
                result = '1' if match_how_supplied and any(word.lower() in match_how_supplied.group(1).lower() for word in target_words_how_supplied) else '0'

                # Append the results to the DataFrame
                results_df = results_df.append({'setid': filename, 'improper_admin': result}, ignore_index=True)

    # Save the results to an Excel file
    results_df.to_excel(output_excel_path, index=False)
    print(f"Results saved to {output_excel_path}")

# Provide the path to your folder containing XML files
xml_folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/XML_9000"

# Specify the output Excel file path
output_excel_path = "/content/drive/MyDrive/jw_jz_4.3.1/9000_improper_admin_flag.xlsx"

# Call the function to check in the folder and save results to Excel
check_how_supplied_in_folder(xml_folder_path, output_excel_path)

 ***display panel - ndc***

***create subset of crosswalk***

In [ ]:
subset_merged_input_crosswalk_df = merged_input_crosswalk_df[['ndc', 'setID']]

In [ ]:
subset_merged_input_crosswalk_df[['setID']] = merged_input_crosswalk_df[['setID']] + ".xml"

<ipython-input-7-96b55e9e1da1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_merged_input_crosswalk_df[['setID']] = merged_input_crosswalk_df[['setID']] + ".xml"


In [ ]:
subset_merged_input_crosswalk_df

***find ndc***

In [ ]:
# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['ndc', 'ndc_flag'])

def rename_and_save_xml_files(xml_folder_path):
    global results_df
    # Iterate through each filename and its corresponding NDCs in the ndc_mapping dictionary
    for i in range(len(subset_merged_input_crosswalk_df)):
        xml_file_path = os.path.join(xml_folder_path, subset_merged_input_crosswalk_df['setID'][i])
        try:
            with open(xml_file_path, 'r', encoding='utf-8') as file:
                xml_content = file.read()
                x = subset_merged_input_crosswalk_df['ndc'][i]
                pattern_display = re.compile(r'<title>.*DISPLAY PANEL.*?</title>\s*<text>(.*?)<title>', re.DOTALL | re.IGNORECASE)
                match_display = pattern_display.search(xml_content)
                result = '1' if match_display and subset_merged_input_crosswalk_df['ndc'][i] in match_display.group(1).lower() else '0'
                # Append result to DataFrame
                results_df = results_df.append({'ndc': x, 'ndc_flag': result}, ignore_index=True)
        except:
            print('fail')

# Provide the path to your folder containing XML files
xml_folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/dailymed_xml"

# Rename and save XML files based on NDC mapping
rename_and_save_xml_files(xml_folder_path)

# Output the DataFrame after the loop is complete
results_df.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/display_ndc_flag.xlsx")


 ***display panel - language***

In [ ]:
def process_sections(xml_content):
    # Define the words to search for in "How Supplied" section
    how_supplied_target_words = ["injection", "infusion"]

    # Define the words to search for in "Display Panel" section
    display_panel_list = ["single intraveneous use", "single intravenous use", "single-dose", "single dose", "single-use", "single use"]

    # Use regex to find the "How Supplied" section content
    how_supplied_pattern = re.compile(r'<title>.*HOW SUPPLIED.*?</title>\s*<text>(.*?)<title>', re.DOTALL | re.IGNORECASE)
    how_supplied_match = how_supplied_pattern.search(xml_content)

    # Use regex to find the "Display Panel" section content
    display_panel_pattern = re.compile(r'<title>.*DISPLAY PANEL.*?</title>\s*<text>(.*?)<title>', re.DOTALL | re.IGNORECASE)
    display_panel_match = display_panel_pattern.search(xml_content)

    # Check "How Supplied" section
    if how_supplied_match:
        how_supplied_content = how_supplied_match.group(1)
        how_supplied_status = 1 if any(word.lower() in how_supplied_content.lower() for word in how_supplied_target_words) else 0
    else:
        how_supplied_status = 0

    # Check "Display Panel" section
    if display_panel_match:
        display_panel_content = display_panel_match.group(1)
        display_panel_status = 1 if any(item.lower() in display_panel_content.lower() for item in display_panel_list) else 0
    else:
        display_panel_status = 0

    return {"display_flag": 1 if how_supplied_status and display_panel_status else 0}

def process_xml_file(xml_file_path):
    # Read the content of the XML file
    with open(xml_file_path, 'r', encoding='utf-8') as file:
        xml_content = file.read()

        # Process both "How Supplied" and "Display Panel" sections
        section_statuses = process_sections(xml_content)

    return {"setid": os.path.basename(xml_file_path), **section_statuses}

def process_xml_folder(folder_path):
    results = []

    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            xml_file_path = os.path.join(folder_path, filename)

            # Process each XML file and store the result in a list
            result = process_xml_file(xml_file_path)
            results.append(result)

    # Convert the list of results to a DataFrame
    df = pd.DataFrame(results)
    return df

# Provide the path to your folder containing XML files
xml_folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/XML_9000"

# Call the function to process the folder
df_results_display = process_xml_folder(xml_folder_path)

# Print the DataFrame
print(df_results_display)

# Save the DataFrame to an Excel file
df_results_display.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/title_split/9000_display_lang_flag.xlsx", index=False)


 ***combine all flags***

In [ ]:
supplied1 = pd.read_excel("/content/drive/MyDrive/jw_jz_4.3.1/supplied_flag.xlsx")
supplied2 = pd.read_excel("/content/drive/MyDrive/jw_jz_4.3.1/title_split/supplied_results.xlsx")
multi = pd.read_excel("/content/drive/MyDrive/jw_jz_4.3.1/multi_flag.xlsx")
admin = pd.read_excel("/content/drive/MyDrive/jw_jz_4.3.1/improper_admin_flag.xlsx")
display = pd.read_excel("/content/drive/MyDrive/jw_jz_4.3.1/display_lang_flag.xlsx")
immuno = pd.read_excel("/content/drive/MyDrive/jw_jz_4.3.1/immunotherapy_flag.xlsx")
ndc = pd.read_excel("/content/drive/MyDrive/jw_jz_4.3.1/display_ndc_flag.xlsx")

In [ ]:
df1 = pd.merge(supplied1, multi, on='setid', how='inner')
df2 = pd.merge(df1, admin, on='setid', how='inner')
df3 = pd.merge(df2, display, on='setid', how='inner')
df5 = pd.merge(df3, immuno, on='setid', how='inner')
#df5 = pd.merge(df4, supplied2, on='setid', how='inner')
#df5 = pd.merge(df4, subset_merged_input_crosswalk_df, left_on='setid', right_on='setID', how='inner')
#df6 = pd.merge(df5, ndc, left_on='ndc', right_on='ndc', how='left')
#df7 = pd.merge(df6, df_input, left_on='ndc', right_on='ndcpackagecode', how='inner')
#df7.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/all_flags1.xlsx", index=False)

In [ ]:
len(df4)

1680

In [ ]:
df5.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/test.xlsx", index=False)

In [ ]:
# Applying conditions to create the new column 'sdc'
df5['sdc'] = 0  # Initialize 'sdc' column with default value 0

# Condition 0: if multi=0 then sdc=0
df5.loc[(df5['immuno_flag'] == 1), 'sdc'] = 1

# Condition 2: if multi=0 then sdc=0
df5.loc[((df5['multi'] == 1) & (df5['immuno_flag'] != 1)), 'sdc'] = 0

# Condition 1: if admin=0 then sdc=0
df5.loc[((df5['improper_admin'] == 1) & (df5['immuno_flag'] != 1)), 'sdc'] = 0

# Condition 3: if supplied = 1 and admin = 0 and multi = 0 then sdc = 1
df5.loc[((df5['supplied_flag'] == 1) & (df5['improper_admin'] == 0) & (df5['multi'] == 0)), 'sdc'] = 1

# Condition 4: if display = 1 and ndc_flag=1 and multi=0 and admin=0 then sdc=1
df5.loc[((df5['display_flag'] == 1) & (df5['multi'] == 0) & (df5['improper_admin'] == 0)), 'sdc'] = 1

df5.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/test.xlsx", index=False)

In [ ]:
# Applying conditions to create the new column 'sdc'
df7['sdc'] = 0  # Initialize 'sdc' column with default value 0

# Condition 0: if multi=0 then sdc=0
df7.loc[(df7['immuno_flag'] == 1), 'sdc'] = 1

# Condition 2: if multi=0 then sdc=0
df7.loc[((df7['multi'] == 1) & (df7['immuno_flag'] == 0)), 'sdc'] = 0

# Condition 1: if admin=0 then sdc=0
df7.loc[((df7['improper_admin'] == 1) & (df7['immuno_flag'] == 0)), 'sdc'] = 0

# Condition 3: if supplied = 1 and admin = 0 and multi = 0 then sdc = 1
df7.loc[((df7['supplied_flag'] == 1) & (df7['improper_admin'] == 0) & (df7['multi'] == 0)), 'sdc'] = 1

# Condition 4: if display = 1 and ndc_flag=1 and multi=0 and admin=0 then sdc=1
df7.loc[((df7['display_flag'] == 1) & (df7['ndc_flag'] == 1) & (df7['multi'] == 0) & (df7['improper_admin'] == 0)), 'sdc'] = 1

df7.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/all_flags.xlsx", index=False)

In [ ]:
df7 = pd.read_excel("/content/drive/MyDrive/jw_jz_4.3.1/test1.xlsx")

In [ ]:
# Applying conditions to create the new column 'sdc'
df7['sdc'] = 0  # Initialize 'sdc' column with default value 0

# Condition 0: if multi=0 then sdc=0
df7.loc[(df7['immuno_flag'] == 1), 'sdc'] = 1

# Condition 2: if multi=0 then sdc=0
df7.loc[((df7['multi'] == 1) & (df7['immuno_flag'] == 0)), 'sdc'] = 0

# Condition 1: if admin=0 then sdc=0
df7.loc[((df7['improper_admin'] == 1) & (df7['immuno_flag'] == 0)), 'sdc'] = 0

# Condition 3: if supplied = 1 and admin = 0 and multi = 0 then sdc = 1
df7.loc[((df7['supplied_flag'] == 1) & (df7['improper_admin'] == 0) & (df7['multi'] == 0)), 'sdc'] = 1

# Condition 4: if display = 1 and ndc_flag=1 and multi=0 and admin=0 then sdc=1
df7.loc[((df7['display_flag'] == 1) & (df7['disp_ndc_flag'] == 1) & (df7['multi'] == 0) & (df7['improper_admin'] == 0)), 'sdc'] = 1

df7.loc[(df7['sdc'] == 1), 'final_flag'] = "Yes"
df7.loc[((df7['sdc'] == 0) & (df7['multi'] == 1) & (df7['supplied_flag'] == 1)), 'final_flag'] = "Unknown"
df7.loc[((df7['sdc'] == 0) & (df7['multi'] == 1) & (df7['display_flag'] == 1)), 'final_flag'] = "Unknown"
df7.loc[((df7['sdc'] == 0) & (df7['multi'] == 1) & (df7['supplied_flag'] == 0)), 'final_flag'] = "No"
df7.loc[((df7['sdc'] == 0) & (df7['multi'] == 1) & (df7['display_flag'] == 0)), 'final_flag'] = "No"
df7.loc[((df7['sdc'] == 0) & (df7['improper_admin'] == 1) & (df7['supplied_flag'] == 0)), 'final_flag'] = "No"
df7.loc[((df7['sdc'] == 0) & (df7['improper_admin'] == 1) & (df7['display_flag'] == 0)), 'final_flag'] = "No"
df7.loc[((df7['sdc'] == 0) & (df7['improper_admin'] == 1) & (df7['display_flag'] == 1)), 'final_flag'] = "Unknown"
df7.loc[((df7['sdc'] == 0) & (df7['improper_admin'] == 1) & (df7['supplied_flag'] == 1)), 'final_flag'] = "Unknown"

df7.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/test2.xlsx", index=False)

# Testing

***How Supplied***

In [ ]:
import re

def extract_how_supplied(xml_file_path):
    # Read the content of the XML file
    with open(xml_file_path, 'r', encoding='utf-8') as file:
        xml_content = file.read()

        # Use regex to find all occurrences of the "How Supplied" section content
        how_supplied_pattern = re.compile(r'<title>[^\n]*HOW SUPPLIED.*?<title>', re.DOTALL | re.IGNORECASE)
        how_supplied_matches = how_supplied_pattern.findall(xml_content)

        if how_supplied_matches:
            # Extract the text of all "How Supplied" sections
            how_supplied_texts = [match.strip() for match in how_supplied_matches]
            return how_supplied_texts
        else:
            return None  # Return None if "How Supplied" section is not found

# Example usage
xml_file_path = "/content/drive/MyDrive/jw_jz_4.3.1/XML_9000/2382setid=3b2dd84e-cee8-4e58-d494-395a65a353a0&.xml"
how_supplied_texts = extract_how_supplied(xml_file_path)
if how_supplied_texts:
    print("Display Panel sections:")
    for text in how_supplied_texts:
        print(text)
else:
    print("How Supplied section not found in the XML file.")


Display Panel sections:
<title>HOW SUPPLIED</title>
               <text>
                  <paragraph>Lidocaine Hydrochloride Injection, USP is supplied as follows:</paragraph>
                  <table width="90%">
                     <col width="50%"/>
                     <col width="50%"/>
                     <thead>
                        <tr>
                           <th align="left" styleCode="Rrule Botrule Toprule " valign="bottom">
                              <content styleCode="bold">Unit of Sale</content>
                           </th>
                           <th align="center" styleCode="Botrule Toprule " valign="bottom">
                              <content styleCode="bold">Concentration</content>
                           </th>
                        </tr>
                     </thead>
                     <tbody>
                        <tr>
                           <td styleCode="Rrule Toprule Botrule " valign="bottom">
                              <p

***Display Panel***

In [ ]:
import re

def extract_how_supplied(xml_file_path):
    # Read the content of the XML file
    with open(xml_file_path, 'r', encoding='utf-8') as file:
        xml_content = file.read()

        # Use regex to find all occurrences of the "How Supplied" section content
        how_supplied_pattern = re.compile(r'<title>.*DISPLAY PANEL.*?</title>\s*(.*?)<title>', re.DOTALL | re.IGNORECASE)
        how_supplied_matches = how_supplied_pattern.findall(xml_content)

        if how_supplied_matches:
            # Extract the text of all "How Supplied" sections
            how_supplied_texts = [match.strip() for match in how_supplied_matches]
            return how_supplied_texts
        else:
            how_supplied_pattern = re.compile(r'<title>.*DISPLAY PANEL.*?</title>\s*(.*?)</document>', re.DOTALL | re.IGNORECASE)
            how_supplied_matches = how_supplied_pattern.findall(xml_content)
            if how_supplied_matches:
              # Extract the text of all "How Supplied" sections
              how_supplied_texts = [match.strip() for match in how_supplied_matches]
              return how_supplied_texts
            else:
              print("How Supplied section not found in the XML file.")

# Example usage
xml_file_path = "/content/drive/MyDrive/jw_jz_4.3.1/dailymed_xml/0dc48e66-6081-421b-952b-4867c9c4c794.xml"
how_supplied_texts = extract_how_supplied(xml_file_path)
if how_supplied_texts:
    print("Display Panel sections:")
    for text in how_supplied_texts:
        print(text)
else:
    print("How Supplied section not found in the XML file.")

Display Panel sections:
<text>
                  <paragraph ID="ID38">
                     <content styleCode="bold">PROCHLORPERAZINE EDISYLATE INJECTION, USP</content>
                  </paragraph>
                  <paragraph>
                     <content styleCode="bold">10 mg/2 mL</content>
                  </paragraph>
                  <paragraph>(5 mg/mL)</paragraph>
                  <paragraph>
                     <content styleCode="bold">NDC </content>23155-<content styleCode="bold">294</content>-41</paragraph>
                  <paragraph>
                     <content styleCode="bold">Rx only</content>
                  </paragraph>
                  <paragraph>
                     <content styleCode="bold">
                        <content styleCode="italics">Twenty Five 2 mL</content> Multiple-Dose Vials</content>
                  </paragraph>
                  <paragraph>
                     <content styleCode="bold">FOR DEEP IM OR IV USE ONLY</content>
        

***Anthony's Method***

In [ ]:
folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/dailymed_xml"

# initialize
SDC_list=[]
NDC_list=[]
sdc_count = 0
mdc_count = 0
sdc_mdc_count = 0
missing_count = 0
missing_dose_count = 0
missing_admin_count = 0
missing_language_count = 0
null_count = 0
no_section_count = 0
non_sdc_count = 0
total_count = 0
how_supplied_count = 0
how_supplied_count0 = 0
sdc = 0
mdc = 0
any_sdc = 0
any_mdc = 0
injection = 0
how_supplied = 0
wrong_application_type = 0
ndc_present = 0
spl_sdc_language = 0
spl_mdc_language = 0
spl_injection_language = 0
immunotherapy = 0
sdc_flag = ""

results_df = pd.DataFrame(columns=['setid','spl_sdc_language','spl_mdc_language','spl_injection_language',
                                    'wrong_application_type', 'injection','any_sdc', 'any_mdc',
                                    'sdc', 'mdc', 'sdc_flag'])

# read XML
for filename in os.listdir(folder_path):
  if filename.endswith('.xml'):
    xml_file_path = os.path.join(folder_path, filename)

    # Read the content of the XML file
    with open(xml_file_path, 'r', encoding='utf-8') as f:
      data = f.read()

      # search title sections
      data_title_sections = data.split("<title>")
      for title_section in data_title_sections:
        if ("single-dose" in title_section.lower() or "single dose" in title_section.lower() or "single-use" in title_section.lower() or "single use" in title_section.lower()):
          spl_sdc_language = 1
        if ("multiple-dose" in title_section.lower() or "mutliple dose" in title_section.lower() or "multiple-use" in title_section.lower() or "multiple use" in title_section.lower() or "multi-dose" in title_section.lower() or "multi dose" in title_section.lower() or "multi-use" in title_section.lower() or "multi use" in title_section.lower()):
          spl_mdc_language = 1
        if "injection" in title_section.lower() or "infusion" in title_section.lower() or "parenteral" in title_section.lower():
          spl_injection_language = 1
        if "immunotherapy" in title_section.lower():
          immunotherapy = 1

        # PRINCIPAL DISPLAY
        if "display panel" in title_section.lower()[0:50]:
          sub_title_section = title_section.split("</title>")
          if "display panel" in sub_title_section[0].lower():
            text2 = sub_title_section[1].lower()

            # single dose container
            if ("single intraveneous use" in text2 or "single intravenous use" in text2 or "single-dose" in text2 or "single dose" in text2 or "single-use" in text2 or "single use" in text2):
              sdc = 1
            # multiple dose container
            if ("multiple-dose" in text2 or "multiple dose" in text2 or "multiple-use" in text2 or "multiple use" in text2 or "multi-dose" in text2 or "multi dose" in text2):
              mdc = 1

        # HOW SUPPLIED
        if "how supplied" in title_section.lower()[0:50]:
          sub_title_section = title_section.split("</title>")

          if "how supplied" in sub_title_section[0].lower():
            text = sub_title_section[1].lower()

            if ("multiple-dose" in text or "multiple dose" in text or "multiple-use" in text or "multiple use" in text or "multi-dose" in text or "multi dose" in text or "multi-use" in text or "multi use" in text):
              any_mdc = 1

            if ("single-dose" in text or "single dose" in text or "single-use" in text or "single use" in text):
              any_sdc = 1

            if "implant" in text or "tablet" in text or "single-patient-use" in text or "single-patient use" in text or "single patient use" in text or "blister" in text or "capsule" in text or "pharmacy bulk" in text or "inhalation" in text or "oral" in text or "gel" in text:
              wrong_application_type = 1

            if "injection" in text or "infusion" in text:
              injection = 1

            # single dose container
            if injection == 1 and wrong_application_type == 0 and any_sdc == 1 and any_mdc == 0:
              sdc = 1
            # multiple dose container
            if spl_sdc_language == 0 and any_mdc == 1:
              mdc = 1

    # IMMUNOTHERAPY SDC correction
    if immunotherapy == 1 and injection == 1:
      sdc = 1
      mdc = 0
    if not (sdc == 1 and mdc == 1):
      if sdc == 1 and injection == 1:
        sdc_flag = "Single Dose Container"
        sdc_count += sdc
      # multiple dose container
      if mdc == 1:
        sdc_flag = "Non Single Dose Container"
        mdc_count += mdc

    # unclear language
    if spl_sdc_language == 0 and spl_mdc_language == 0:
      sdc_flag = "Missing Single-Dose and Multiple-Dose Language"
      missing_dose_count += 1
    if spl_injection_language == 0:
      sdc_flag = "Missing Injection, Infusion and Parenteral Language"
      missing_admin_count += 1
    if spl_sdc_language == 0 and spl_mdc_language == 0 and spl_injection_language == 0:
      sdc_flag = "Missing Single-Dose and Multiple-Dose Language and Injection, Infusion and Parenteral Language"
      missing_language_count += 1

    # append to lists
    results_df = results_df._append({'setid': filename, 'spl_sdc_language': spl_sdc_language, 'spl_mdc_language': spl_mdc_language, 'spl_injection_language': spl_injection_language,
                                    'wrong_application_type':wrong_application_type, 'injection':injection, 'any_sdc':any_sdc, 'any_mdc':any_mdc,
                                    'sdc':sdc, 'mdc':mdc, 'sdc_flag':sdc_flag}, ignore_index=True)
    SDC_list.append(sdc_flag)

# convert list to dataframe
SDC_df = pd.DataFrame(SDC_list)
SDC_df.columns = ["SDC Flag"]

# print counts
print("single dose count:",sdc_count)
print("multiple dose count:",mdc_count)
print("null count:",null_count)
print("missing dose language:",missing_dose_count)
print("missing administration language:",missing_admin_count)
print("missing dose and administration language:",missing_language_count)
print("identified count:",sdc_count+mdc_count+null_count+missing_dose_count+missing_admin_count+missing_language_count)
print("total count:",total_count)




single dose count: 1624
multiple dose count: 0
null count: 0
missing dose language: 0
missing administration language: 0
missing dose and administration language: 0
identified count: 1624
total count: 0


In [ ]:
results_df.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/results_df.xlsx")

# SDC Classification - Title Split

 ***how supplied***

In [ ]:
folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/dailymed_xml/june_2024"
results_df = pd.DataFrame(columns=['setid','sdc_flag'])
z=0
for filename in os.listdir(folder_path):
  if filename.endswith('.xml'):
    xml_file_path = os.path.join(folder_path, filename)

    with open(xml_file_path, 'r', encoding='utf-8') as f:
        data = f.read()
        z=z+1
        print(z)

        how_supplied_title = "doesn't exist"
        data_title_sections = data.split("<title>")
        for title_section in data_title_sections:
          if "how supplied" in title_section.lower()[0:50]:
            sub_title_section = title_section.split("</title>")
            if "how supplied" in sub_title_section[0].lower():
              text = sub_title_section[1].lower()

              how_supplied_title = "exists"
              # Define the words to search for in "How Supplied" section
              injection_words = ["injection", "infusion"]
              single_words = ["single-dose", "single dose", "single-use", "single use"]
              improper_words = ["implant", "tablet","single-patient-use", "single-patient use","single patient use", "blister","capsule", "pharmacy bulk", "inhalation", "oral", "gel"]

              if any(word.lower() in text.lower() for word in injection_words) and any(item.lower() in text.lower() for item in single_words):
                sdc_flag=1
              else:
                sdc_flag=0
              if any(word.lower() in text.lower() for word in improper_words):
                improper_flag=1
              else:
                improper_flag=0
              results_df = results_df._append({"setid": filename, "sdc_flag": sdc_flag, "how supplied title": how_supplied_title, "improper_flag":improper_flag}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
results_df.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/title_split/june_2024_supplied_results.xlsx")

 ***immunotherapy***

In [ ]:
def search_immunotherapy_in_folder(folder_path, output_excel_path):
    # Define the words to search for
    target_word_immunotherapy = "immunotherapy"
    target_words_how_supplied = ["injection", "infusion"]

    # Create an empty DataFrame to store the results
    results_df = pd.DataFrame(columns=['setid', 'immuno_flag'])

    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            xml_file_path = os.path.join(folder_path, filename)

            # Open and read the content of the XML file
            with open(xml_file_path, 'r', encoding='utf-8') as file:
                xml_content = file.read()

                # Check if immunotherapy word exists in the content
                found_immunotherapy = 1 if target_word_immunotherapy.lower() in xml_content.lower() else 0

                # Use regex to find the "How Supplied" section content
                data_title_sections = xml_content.split("<title>")
                for title_section in data_title_sections:
                  if "how supplied" in title_section.lower()[0:50]:
                    sub_title_section = title_section.split("</title>")
                    if "how supplied" in sub_title_section[0].lower():
                      text = sub_title_section[1].lower()

                      # Check if the section was found and if it contains the target words for "How Supplied"
                      found_how_supplied = 1 if any(word.lower() in text for word in target_words_how_supplied) else 0

                      # Combine the results for both immunotherapy and how supplied
                      found_combined = 1 if found_immunotherapy and found_how_supplied else 0

                      # Append the results to the DataFrame
                      results_df = results_df._append({'setid': filename, 'immuno_flag': found_combined}, ignore_index=True)

    # Save the results to an Excel file
    results_df.to_excel(output_excel_path, index=False)
    print(f"Results saved to {output_excel_path}")

# Provide the path to your folder containing XML files
xml_folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/dailymed_xml/june_2024"

# Specify the output Excel file path
output_excel_path = "/content/drive/MyDrive/jw_jz_4.3.1//title_split/june_2024_immunotherapy_results.xlsx"

# Call the function to search in the folder and save results to Excel
search_immunotherapy_in_folder(xml_folder_path, output_excel_path)

Results saved to /content/drive/MyDrive/jw_jz_4.3.1//title_split/june_2024_immunotherapy_results.xlsx


***multi-dose***

In [ ]:
folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/dailymed_xml/june_2024"
results_df = pd.DataFrame(columns=['setid','multi_flag'])
z=0
for filename in os.listdir(folder_path):
  if filename.endswith('.xml'):
    xml_file_path = os.path.join(folder_path, filename)

    with open(xml_file_path, 'r', encoding='utf-8') as f:
        data = f.read()
        z=z+1
        print(z)

        how_supplied_title = "doesn't exist"
        data_title_sections = data.split("<title>")
        for title_section in data_title_sections:
          if "how supplied" in title_section.lower()[0:50]:
            sub_title_section = title_section.split("</title>")
            if "how supplied" in sub_title_section[0].lower():
              text = sub_title_section[1].lower()

              how_supplied_title = "exists"
              # Define the words to search for in "How Supplied" section
              target_words = ["multiple-dose", "multiple dose", "multiple-use", "multiple use", "multi-dose", "multi dose", "multi-use", "multi use"]

              if any(word.lower() in text.lower() for word in target_words):
                multi_flag=1
              else:
                multi_flag=0

              results_df = results_df._append({"setid": filename, "multi_flag":multi_flag}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
results_df.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/title_split/june_2024_multi_flag_disp.xlsx", index=False)

In [ ]:
folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/dailymed_xml/june_2024"
results_df = pd.DataFrame(columns=['setid','multi_flag'])
z=0
for filename in os.listdir(folder_path):
  if filename.endswith('.xml'):
    xml_file_path = os.path.join(folder_path, filename)

    with open(xml_file_path, 'r', encoding='utf-8') as f:
        data = f.read()
        z=z+1
        print(z)

        how_supplied_title = "doesn't exist"
        data_title_sections = data.split("<title>")
        for title_section in data_title_sections:
          if "display panel" in title_section.lower()[0:50]:
            sub_title_section = title_section.split("</title>")
            if "display panel" in sub_title_section[0].lower():
              text = sub_title_section[1].lower()

              how_supplied_title = "exists"
              # Define the words to search for in "How Supplied" section
              target_words = ["multiple-dose", "multiple dose", "multiple-use", "multiple use", "multi-dose", "multi dose", "multi-use", "multi use"]

              if any(word.lower() in text.lower() for word in target_words):
                multi_flag=1
              else:
                multi_flag=0

              results_df = results_df._append({"setid": filename, "multi_flag":multi_flag}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


***Display Panel***

In [ ]:
# Save the DataFrame to an Excel file
results_df1.to_excel("/content/drive/MyDrive/jw_jz_4.3.1/title_split/june_2024_display_flag1.xlsx", index=False)

In [ ]:
folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/dailymed_xml/june_2024"
results_df = pd.DataFrame(columns=['setid','injection_flag'])
z=0
for filename in os.listdir(folder_path):
  if filename.endswith('.xml'):
    xml_file_path = os.path.join(folder_path, filename)

    with open(xml_file_path, 'r', encoding='utf-8') as f:
        data = f.read()
        z=z+1
        print(z)

        how_supplied_title = "doesn't exist"
        data_title_sections = data.split("<title>")
        for title_section in data_title_sections:
          if "how supplied" in title_section.lower()[0:50]:
            sub_title_section = title_section.split("</title>")
            if "how supplied" in sub_title_section[0].lower():
              text = sub_title_section[1].lower()

              how_supplied_title = "exists"
              # Define the words to search for in "How Supplied" section
              injection_words = ["injection", "infusion"]

              if any(word.lower() in text.lower() for word in injection_words):
                injection_flag=1
              else:
                injection_flag=0
              results_df = results_df._append({"setid": filename, "injection_flag":injection_flag}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
folder_path = "/content/drive/MyDrive/jw_jz_4.3.1/dailymed_xml/june_2024"
results_df1 = pd.DataFrame(columns=['setid','single_flag'])
z=0
for filename in os.listdir(folder_path):
  if filename.endswith('.xml'):
    xml_file_path = os.path.join(folder_path, filename)

    with open(xml_file_path, 'r', encoding='utf-8') as f:
        data = f.read()
        z=z+1
        print(z)

        how_supplied_title = "doesn't exist"
        data_title_sections = data.split("<title>")
        for title_section in data_title_sections:
          if "display panel" in title_section.lower()[0:50]:
            sub_title_section = title_section.split("</title>")
            if "display panel" in sub_title_section[0].lower():
              text = sub_title_section[1].lower()

              how_supplied_title = "exists"
              # Define the words to search for in "How Supplied" section
              single_words = ["single-dose", "single dose", "single-use", "single use"]

              if any(item.lower() in text.lower() for item in single_words):
                single_flag=1
              else:
                single_flag=0

              results_df1 = results_df1._append({"setid": filename, "single_flag":single_flag}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
